## 학습~ 이미지 생성입니다
#### 경로 설정은 따로 잘해주세요

In [ ]:
# styleGAN2 - ADA 모델은 텐서플로우 1버전에서만 작동
%tensorflow_version 1.x

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 코드 다운로드
%cd /content/
!git clone https://github.com/dvschultz/stylegan2-ada.git
!mkdir downloads
!mkdir datasets

In [ ]:
# 포케못 이미지 데이터
https://www.kaggle.com/kvpratama/pokemon-images-dataset

In [ ]:
dataset_name = "pokemon-256"

In [ ]:
# 압축파일해제
import zipfile
path = "/content/drive/MyDrive/last_project/stylegan/"
dataset = dataset_name + ".zip"
local_path = "/content/"
file_name = path + dataset
with zipfile.ZipFile(file_name, 'r') as zip:
   #zip.printdir()
   print('Extracting all the files now...') 
   zip.extractall(local_path) 
   print('Done!')

In [ ]:
# 리사이즈
# 포케못 가져오신거면 이거 안돌려도됩니당
import os
import glob
from PIL import Image

files = glob.glob('/content/pokemon-256/*.jpg')

for f in files:
    img = Image.open(f)
    img = img.convert('RGB')
    img_resize = img.resize((int(256), int(256)))
    title, ext = os.path.splitext(f)
    img_resize.save(title + ext)



In [ ]:
# 불러온 이미지들을 텐서플로우에서 사용할 수 있게 포맷(tfrecords)형식으로 변환
%cd /content/

#update this to the path to your image folder
dataset_folder_name = 'pokemon_jpg/pokemon_jpg' # name of zip file may be different from folder name it extracted to
dataset_path = "/content/" + dataset_folder_name

#you don't need to edit anything here
%cd /content/stylegan2-ada
!python dataset_tool.py create_from_images /content/datasets/{dataset_name} {dataset_path}

In [ ]:
# 파라미터 확인
%cd /content/stylegan2-ada
!python train.py --help

In [ ]:
# 넘파이 버전이 텐서플로우 버전보다 높아서 학습을 넘파이 1버전으로 안하면 오류뜹니다
# 삭제 하시고 다시 재설치
!pip uninstall numpy
!pip install numpy==1.16.4

In [ ]:
# 학습시작
# tick이 50으로 디폴트 되어있는데 찾아서 설정 하시거나 10개 정도까지 해도 괜찮았습니다
# output directory
output_dir = '/content/drive/MyDrive/Colab\ Notebooks/results/' + dataset_name + "/"

# config
config = "auto"

# gamma
gamma = 1

#how often should the model generate samples and a .pkl file
snapshot_count = 1

#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False

#metrics? 
metric_list = None
#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
resume_from = "ffhq256" # 파라미터를 확인 해보시면 알겠지만 사진 사이즈에 따라 값 변경해줘야하는것 같습니다

# make sure there is no space in the resume path. if there is any, use a backslash character to escape
# resume_from = '/content/drive/MyDrive/Colab\ Notebooks/results/pokemon-256/00001-pokemon-256-mirror-auto1-gamma1-resumecustom/network-snapshot-000036.pkl'

#don't edit this unless you know what you're doing :)
!python train.py --outdir={output_dir} \
                 --cfg={config} \
                 --snap={snapshot_count} \
                 --data=/content/datasets/{dataset_name} \
                 --mirror={mirrored} --mirrory={mirroredY} \
                 --gamma={gamma} \
                 --metrics={metric_list} \
                 --resume={resume_from}

In [ ]:
import cv2
from os import listdir
from os.path import isfile, join
from google.colab import files
from PIL import Image, ImageDraw
import numpy as np
from tqdm import tqdm
import imageio
import os

In [ ]:
#result_path를 학습 과정 이미지들을 저장한 폴더 경로 바꿔주기
# choose where you've saved results
# result_path = '/content/drive/MyDrive/Colab Notebooks/results/pokemon-256/00001-pokemon-256-mirror-auto1-gamma1-resumecustom/'
result_path = '/content/drive/MyDrive/last_project/stylegan/pokemon-256/00000-pokemon-256-mirror-auto1-gamma1-resumeffhq256/'
image_files = [f for f in listdir(result_path)]
image_files = [f for f in image_files if '.jpg' in f]

image_files.sort()

print(image_files)

In [ ]:
images = []

# change settings width, width_images, height, height_images
width = 256
width_images = 8
width_offset = 0 * width
end_width = max(1920, width_images * width) 

height = 256
height_images = 4
height_offset = 0 * height
end_height = max(1280, height_images * height)


for f in tqdm(image_files[:]):
  name = "{}{}".format(result_path, f)
  print(name)
  img = Image.open(name)
  img = img.crop((0, 0, end_width+ width_offset, end_height + height_offset))
  img = img.resize((1920,1080))
  images.append(img)

In [ ]:
# video file 저장할 폴더 생성
try:
  os.mkdir("/content/out/")
except:
  pass

In [ ]:
# change path to where you want to save video
video_output_path = '/content/out/'
video_name = 'timelapse_movie.mp4'
movie_name = video_output_path + video_name

with imageio.get_writer(movie_name, mode='I') as writer:
    for image in tqdm(list(images)):
        writer.append_data(np.array(image))

In [ ]:
#영상 다운받고싶으시면 다운
files.download(movie_name) 

In [ ]:
%cd /content/stylegan2-ada
import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
from io import BytesIO
import IPython.display
import numpy as np
from math import ceil
from PIL import Image, ImageDraw
import imageio
import os
import pickle
from google.colab import files

In [ ]:
# 훈련시킨 모델 pkl 파일 경로를 network_pkl로 지정
dnnlib.tflib.init_tf()

# change path
network_pkl = '/content/drive/MyDrive/last_project/stylegan/pokemon-256/00000-pokemon-256-mirror-auto1-gamma1-resumeffhq256/network-snapshot-000040.pkl'
 
print('Loading networks from "%s"...' % network_pkl)
with dnnlib.util.open_url(network_pkl) as fp:
    _G, _D, Gs = pickle.load(fp)
noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

In [ ]:
# 이미지를 생성하는데 필요한 함수 정의
def generate_images_in_w_space(dlatents, truncation_psi):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    Gs_kwargs.truncation_psi = truncation_psi
    dlatent_avg = Gs.get_var('dlatent_avg') # [component]

    imgs = []
    for row, dlatent in log_progress(enumerate(dlatents), name = "Generating images"):
        #row_dlatents = (dlatent[np.newaxis] - dlatent_avg) * np.reshape(truncation_psi, [-1, 1, 1]) + dlatent_avg
        dl = (dlatent-dlatent_avg)*truncation_psi   + dlatent_avg
        row_images = Gs.components.synthesis.run(dlatent,  **Gs_kwargs)
        imgs.append(PIL.Image.fromarray(row_images[0], 'RGB'))
    return imgs       

def generate_images(zs, truncation_psi):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if not isinstance(truncation_psi, list):
        truncation_psi = [truncation_psi] * len(zs)
        
    imgs = []
    for z_idx, z in log_progress(enumerate(zs), size = len(zs), name = "Generating images"):
        Gs_kwargs.truncation_psi = truncation_psi[z_idx]
        noise_rnd = np.random.RandomState(1) # fix noise
        tflib.set_vars({var: noise_rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(z, None, **Gs_kwargs) # [minibatch, height, width, channel]
        imgs.append(PIL.Image.fromarray(images[0], 'RGB'))
    return imgs

def generate_zs_from_seeds(seeds):
    zs = []
    for seed_idx, seed in enumerate(seeds):
        rnd = np.random.RandomState(seed)
        z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        zs.append(z)
    return zs

# Generates a list of images, based on a list of seed for latent vectors (Z), and a list (or a single constant) of truncation_psi's.
def generate_images_from_seeds(seeds, truncation_psi):
    return generate_images(generate_zs_from_seeds(seeds), truncation_psi)

def saveImgs(imgs, location):
  for idx, img in log_progress(enumerate(imgs), size = len(imgs), name="Saving images"):
    file = location+ str(idx) + ".png"
    img.save(file)

def imshow(a, format='png', jpeg_fallback=True):
  a = np.asarray(a, dtype=np.uint8)
  str_file = BytesIO()
  PIL.Image.fromarray(a).save(str_file, format)
  im_data = str_file.getvalue()
  try:
    disp = IPython.display.display(IPython.display.Image(im_data))
  except IOError:
    if jpeg_fallback and format != 'jpeg':
      print ('Warning: image was too large to display in format "{}"; '
             'trying jpeg instead.').format(format)
      return imshow(a, format='jpeg')
    else:
      raise
  return disp

def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

        
def clamp(x, minimum, maximum):
    return max(minimum, min(x, maximum))
    
def drawLatent(image,latents,x,y,x2,y2, color=(255,0,0,100)):
  buffer = PIL.Image.new('RGBA', image.size, (0,0,0,0))
   
  draw = ImageDraw.Draw(buffer)
  cy = (y+y2)/2
  draw.rectangle([x,y,x2,y2],fill=(255,255,255,180), outline=(0,0,0,180))
  for i in range(len(latents)):
    mx = x + (x2-x)*(float(i)/len(latents))
    h = (y2-y)*latents[i]*0.1
    h = clamp(h,cy-y2,y2-cy)
    draw.line((mx,cy,mx,cy+h),fill=color)
  return PIL.Image.alpha_composite(image,buffer)
             
  
def createImageGrid(images, scale=0.25, rows=1):
   w,h = images[0].size
   w = int(w*scale)
   h = int(h*scale)
   height = rows*h
   cols = ceil(len(images) / rows)
   width = cols*w
   canvas = PIL.Image.new('RGBA', (width,height), 'white')
   for i,img in enumerate(images):
     img = img.resize((w,h), PIL.Image.ANTIALIAS)
     canvas.paste(img, (w*(i % cols), h*(i // cols))) 
   return canvas

def convertZtoW(latent, truncation_psi=0.7, truncation_cutoff=9):
  dlatent = Gs.components.mapping.run(latent, None) # [seed, layer, component]
  dlatent_avg = Gs.get_var('dlatent_avg') # [component]
  for i in range(truncation_cutoff):
    dlatent[0][i] = (dlatent[0][i]-dlatent_avg)*truncation_psi + dlatent_avg
    
  return dlatent

def interpolate(zs, steps):
   out = []
   for i in range(len(zs)-1):
    for index in range(steps):
     fraction = index/float(steps) 
     out.append(zs[i+1]*fraction + zs[i]*(1-fraction))
   return out

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
#truncation_psi(0~1) 값 변경해서 이미지 다양하게 조절가능
seeds = np.random.randint((2**32 - 1), size=9)
imshow(createImageGrid(generate_images_from_seeds(seeds, truncation_psi=0.9), 0.7 , 3))

In [ ]:
# 결과 이미지를 영상으로 만듦
video_output_path = '/content/out/'
video_name = 'interpolation_movie.mp4'
movie_name = video_output_path + video_name

In [ ]:
# number_of_steps와 trunc_psi 값 조절해보세용
# interpolate between 10 seeds (last one back to first to loop)
number_of_steps = 10
trunc_psi = 0.9

seeds = list(np.random.randint((2**32) - 1, size=9))
seeds = seeds + [seeds[0]]
zs = generate_zs_from_seeds(seeds)


imgs = generate_images(interpolate(zs,number_of_steps), trunc_psi)

with imageio.get_writer(movie_name, mode='I') as writer:
    for image in log_progress(list(imgs), name = "Creating animation"):
        writer.append_data(np.array(image))

In [ ]:
#영상 다운받고싶으시면 다운
files.download(movie_name) 

In [ ]:
# project into w space
number_of_steps = 10
trunc_psi = 0.5

video_name = 'interpolation_ws_movie.mp4'
movie_name = video_output_path + video_name

dls = []
for z in zs:
  dls.append(convertZtoW(z ,truncation_psi=trunc_psi))

imgs = generate_images_in_w_space(interpolate(dls,number_of_steps), trunc_psi)

with imageio.get_writer(movie_name, mode='I') as writer:
    for image in log_progress(list(imgs), name = "Creating animation"):
        writer.append_data(np.array(image))

In [ ]:
#영상 다운받고싶으시면 다운
files.download(movie_name) 